In [1]:
import logging
import importlib
import TwitterFactCheck
import TwitterAPI
import pandas as pd
import altair as alt
import re
import logging
import numpy as np
from typing import *
from datetime import *
import json
import requests
from bs4 import BeautifulSoup
from multiprocessing.dummy import Process
import traceback
from collections import Counter
importlib.reload(TwitterFactCheck)
importlib.reload(TwitterAPI)
logging.basicConfig(level=logging.INFO)

In [2]:
%%time
api = TwitterFactCheck.TwitterFactCheck("Backbackup.json")
start_time, end_time = "20220101", "20220401"
status = dict()

CPU times: user 1.47 ms, sys: 151 µs, total: 1.62 ms
Wall time: 4.18 ms


In [3]:
def fetch_interaction(u1, u2, start_time, end_time):
    data = list()
    if u1.name > u2.name: # compare based on user id
        u1, u2 = u2, u1
    query = " OR ".join(["(@{u1} @{u2})", "(from:{u1} @{u2})", "(from:{u2} @{u1})", "(from:{u2} to:{u1})", "(from:{u1} to:{u2})"])
    query = query.format(u1=u1.username, u2=u2.username)
    tweets = api.search_tweets(query, start_time=start_time, end_time=end_time)
    tweets["u1"], tweets["u2"] = u1.name, u2.name
    tweets["author1"], tweets["author2"] = u1.username, u2.username
    data.append(tweets)
    return pd.concat(data) if data else pd.DataFrame()

In [29]:
df_method_tweets.to_csv("Data/Network/NetworkTweets.csv", sep="\t", index_label="id")

In [4]:
def fetch_sus_content(status: dict, start_time, end_time, method: str):
    df_method_tweets = pd.read_csv("Data/Network/NetworkTweets.csv", sep="\t", index_col="id")
    df_method_users = pd.read_csv("Data/Network/NetworkUsers.csv", index_col="user_id", sep="\t")
    df_method_tweets = df_method_tweets[df_method_tweets.method==method]
    df_method_users = df_method_users[df_method_users.method==method]
    iteration = status["iteration"] = df_method_users.iteration.max() + 1
    status["df_method_users"], status["df_method_tweets"] = df_method_users, df_method_tweets
    status["status"] = "prepare"
    df_sus_users = pd.Series(Counter(df_method_tweets.author_id))
    df_sus_users = df_sus_users[df_sus_users>5]
    df_sus_users = df_sus_users[df_sus_users>df_sus_users.quantile(0.8)]
    status["df_new_users"] = df_sus_users
    df_new_users = dict()
    status["status"] = "fetching {} new users".format(len(df_sus_users))
    for author_id in df_sus_users.index.astype(int):
        if author_id not in df_method_users.index:
            info = api.search_user(author_id)
            if info:
                df_new_users[author_id] = info
    status["df_new_users"] = df_new_users = pd.DataFrame(df_new_users).T.dropna(how="all").sort_index()
    status["df_new_tweets"] = df_new_tweets = list()
    status["status"] = "fetching tweets"
    for i in range(len(df_new_users)):
        status["i"] = i
        for t in range(0, len(df_method_users)):
            status["t"] = t
            df_new_tweets.append(fetch_interaction(df_new_users.iloc[i], df_method_users.iloc[t], start_time, end_time))

        for j in range(i+1, len(df_new_users)):
            df_new_tweets.append(fetch_interaction(df_new_users.iloc[i], df_new_users.iloc[j], start_time, end_time))
            status["j"] = j

    df_new_tweets = pd.concat(df_new_tweets).set_index(["id"])
    df_new_tweets.text = df_new_tweets.text.replace("\s+", " ")
    df_new_tweets["content"] = df_new_tweets.text.str.replace("(@[\w|\d]+|\#[\w|\d]+|https\S+)", " ")
    for s in ["\s{2,}", "RT:\s?", "^\s+\$"]:
        df_new_tweets["content"] = df_new_tweets["content"].str.replace(s, "")
    status["status"] = "fetching complete"
    for col in ["author_id"]:
        df_new_tweets[col] = df_new_tweets[col].astype(int)
    status["df_method_tweets"] = df_new_tweets
    df_new_tweets["method"], df_new_users["method"] = method, method
    df_new_tweets["iteration"] = df_new_users["iteration"] = iteration
    df_new_users.to_csv("Data/tmp/{}_{}_{}_NetworkUsers.csv".format(i, t, j), sep="\t", index_label="user_id")
    df_new_tweets.to_csv("Data/tmp/{}_{}_{}_NetworkTweets.csv".format(i, t, j), sep="\t", index_label="id")

In [5]:
status = dict()
thread = Process(target=fetch_sus_content, kwargs={"status": status, "start_time": start_time, "end_time": end_time, "method": "URL"})
thread.start()

In [6]:
thread.is_alive()

True